SI 671 Project 

In [18]:
import pandas as pd
import numpy as np 
import json
import sqlite3
business_path = "Data/yelp_academic_dataset_business.json"
checkin_path = "Data/yelp_academic_dataset_checkin.json"
review_path = "Data/yelp_academic_dataset_review.json"
tip_path = 'Data/yelp_academic_dataset_tip.json'
user_path = 'Data/yelp_academic_dataset_user.json'

# load data

In [19]:
def json_to_df(path):
    with open(path) as data_file:
        data = []
        for line in data_file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    return df

In [20]:
df_business = json_to_df(business_path)
df_checkin = json_to_df(checkin_path)
df_review = json_to_df(review_path)
df_tip = json_to_df(tip_path)
df_user = json_to_df(user_path)

### check-in data

In [21]:
df_checkin.dtypes

business_id    object
date           object
dtype: object

In [22]:
convert_dict = {'business_id': "string",
                'date':"string"
                }
 
df_checkin = df_checkin.astype(convert_dict)

In [23]:
check_in_schema="""
DROP TABLE IF EXISTS "checkin";

CREATE TABLE "checkin" (
    "business_id" VARCHAR PRIMARY KEY NOT NULL,
    "date" VARCHAR
    );
"""

### business data

In [24]:
df_business.dtypes

business_id      object
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
stars           float64
review_count      int64
is_open           int64
attributes       object
categories       object
hours            object
dtype: object

In [25]:
convert_dict = {'business_id': "string",
                'name': "string",
                'address':"string",
                'city':"string",
                'state' : "string",
                'postal_code' :"string",
                'attributes' :"string",
                'categories':"string",
                'hours' :"string"
                }
 
df_business = df_business.astype(convert_dict)

In [26]:
businesses_schema="""
DROP TABLE IF EXISTS "business";

CREATE TABLE "business" (
    "business_id" VARCHAR PRIMARY KEY NOT NULL,
    "name" VARCHAR,
    "address" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "postal_code" VARCHAR,
    "latitude" FLOAT,
    "longitude" FLOAT,
    "stars" FLOAT,
    "review_count" INTEGER,
    "is_open" BOOLEAN,
    "attributes" VARCHAR,
    "categories" VARCHAR,
    "hours" VARCHAR
    );
"""

### user data

In [27]:
df_user.dtypes

user_id                object
name                   object
review_count            int64
yelping_since          object
useful                  int64
funny                   int64
cool                    int64
elite                  object
friends                object
fans                    int64
average_stars         float64
compliment_hot          int64
compliment_more         int64
compliment_profile      int64
compliment_cute         int64
compliment_list         int64
compliment_note         int64
compliment_plain        int64
compliment_cool         int64
compliment_funny        int64
compliment_writer       int64
compliment_photos       int64
dtype: object

In [28]:
convert_dict = {'user_id': "string",
                'name':"string",
                'yelping_since':"string",
                'elite':"string",
                'friends':"string",
                }
 
df_user = df_user.astype(convert_dict)

In [29]:
users_schema = """
DROP TABLE IF EXISTS "users";

CREATE TABLE "users" (
    "user_id" VARCHAR PRIMARY KEY NOT NULL,
    "name" VARCHAR,
    "review_count" INTEGER,
    "yelping_since" TIMESTAMP,
    "useful" INTEGER,
    "funny" INTEGER,
    "cool" INTEGER,
    "elite" VARCHAR,
    "friends" VARCHAR,
    "fans" INTEGER,
    "average_stars" FLOAT,
    "compliment_hot" INTEGER,
    "compliment_more" INTEGER, 
    "compliment_profile" INTEGER,
    "compliment_cute" INTEGER,
    "compliment_list" INTEGER,
    "compliment_note" INTEGER,
    "compliment_plain" INTEGER,
    "compliment_cool" INTEGER,
    "compliment_funny" INTEGER,
    "compliment_writer" INTEGER,
    "compliment_photos" INTEGER
);
"""

### reviews

In [30]:
df_review.dtypes

review_id       object
user_id         object
business_id     object
stars          float64
useful           int64
funny            int64
cool             int64
text            object
date            object
dtype: object

In [31]:
convert_dict = {'review_id': "string",
                'user_id':"string",
                'business_id':"string",
                'text':"string",
                'date':"string",
                }
 
df_review = df_review.astype(convert_dict)

In [32]:
reviews_schema = """
DROP TABLE IF EXISTS "reviews";

CREATE TABLE "reviews" (
    "review_id" VARCHAR PRIMARY KEY,
    "user_id" VARCHAR,
    "business_id" VARCHAR,
    "stars" FLOAT,
    "useful" INTEGER,
    "funny" INTEGER,
    "cool" INTEGER,
    "text"  VARCHAR,
    "date" VARCHAR,
    
    
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (business_id) REFERENCES businesses(business_id)
);
"""

### tips

In [33]:
df_tip.dtypes

user_id             object
business_id         object
text                object
date                object
compliment_count     int64
dtype: object

In [34]:
convert_dict = {'user_id':"string",
                'business_id':"string",
                'text':"string",
                'date':"string",
                }
 
df_tip = df_tip.astype(convert_dict)

In [35]:
tips_schema = """
DROP TABLE IF EXISTS "tips";

CREATE TABLE "tips" (
    "user_id" VARCHAR,
    "business_id" VARCHAR,
    "text"  VARCHAR,
    "date" VARCHAR,
    "compliment_count" INTEGER,
    
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (business_id) REFERENCES businesses(business_id)
);
"""

In [36]:
conn = sqlite3.connect('yelp_dataset_business.db') 
c = conn.cursor()
c.executescript(businesses_schema)
conn.commit()

In [37]:
df_business.to_sql(name='business', con=conn, if_exists='append',index=False)
conn.close()

In [38]:
conn = sqlite3.connect('yelp_dataset_checkin.db') 
c = conn.cursor()
c.executescript(check_in_schema)
conn.commit()

In [39]:
df_checkin.to_sql(name='checkin', con=conn, if_exists='append',index=False)
conn.close()

In [40]:
conn = sqlite3.connect('yelp_dataset_users.db') 
c = conn.cursor()
c.executescript(users_schema)
conn.commit()

In [41]:
df_user.to_sql(name='users', con=conn, if_exists='append',index=False)
conn.close()

In [42]:
conn = sqlite3.connect('yelp_dataset_reviews.db') 
c = conn.cursor()
c.executescript(reviews_schema)
conn.commit()

In [43]:
df_review.to_sql(name='reviews', con=conn, if_exists='append',index=False)
conn.close()

In [44]:
conn = sqlite3.connect('yelp_dataset_tips.db') 
c = conn.cursor()
c.executescript(tips_schema)
conn.commit()

In [45]:
df_tip.to_sql(name='tips', con=conn, if_exists='append',index=False)
conn.close()

In [50]:
conn.close()

In [49]:
conn = sqlite3.connect('yelp_dataset_tips.db') 
new = pd.read_sql("""SELECT * FROM tips""",conn )
new.shape

(908915, 5)